In [325]:
import pandas as pd

In [326]:
df = pd.read_csv(r'C:\Users\mw865\Desktop\NTI_Project\data\raw\egypt_data.csv')

df.head(5)

,user,rating,review
0,مستخدم Google,1,سحبو للمره الثانيه رصيد من البطاقه و لغو الطلب...
1,مستخدم Google,1,رائع
2,مستخدم Google,5,طلبات ممتاز
3,مستخدم Google,5,🤤💗😉
4,مستخدم Google,1,والله ما انصح بطلبات ما عندهم مصدقيه بالوقت بس...


In [327]:
df.isna().sum()

user      0
rating    0
review    8
dtype: int64

In [328]:
df.duplicated().sum()

np.int64(27886)

In [329]:
df = df.drop_duplicates().reset_index(drop=True)
df = df.dropna().reset_index(drop=True)
df = df.drop(labels="user" , axis=1)

In [330]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47810 entries, 0 to 47809
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rating  47810 non-null  int64 
 1   review  47810 non-null  object
dtypes: int64(1), object(1)
memory usage: 747.2+ KB


In [331]:
def label_sentiment(rating):
    if rating <= 2:
        return "negative"
    else:
        return "positive"

df['sentiment'] = df['rating'].apply(label_sentiment)
df['sentiment'].value_counts()

sentiment
positive    30465
negative    17345
Name: count, dtype: int64

In [332]:
df.head()

,rating,review,sentiment
0,1,سحبو للمره الثانيه رصيد من البطاقه و لغو الطلب...,negative
1,1,رائع,negative
2,5,طلبات ممتاز,positive
3,5,🤤💗😉,positive
4,1,والله ما انصح بطلبات ما عندهم مصدقيه بالوقت بس...,negative


In [333]:
import re

def normalize_arabic(text):
  # Remove diacritics
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text)
  # Normalize letters variations
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("ڤ", "ف", text)
    text = re.sub("چ", "ج", text)
    text = re.sub("پ", "ب", text)
    text = re.sub("ڜ", "ش", text)
    text = re.sub("ڪ", "ك", text)
    text = re.sub("ڧ", "ق", text)
    text = re.sub("ٱ", "ا", text)
    
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

df['review'] = df['review'].apply(normalize_arabic)

In [334]:
# !pip install emoji -q

In [335]:
import emoji

def handle_emojis(text, mode='remove'):
    if mode == 'remove':
        return emoji.replace_emoji(text, '')
    
df['review'] = df['review'].apply(handle_emojis)
    

In [336]:
# import re

# # Keep only Arabic letters and spaces in each review
df['review'] = df['review'].apply(lambda x: re.sub(r'[^\u0600-\u06FF\s]', '', x))

df = df[df['review'].str.strip() != ""]
df = df.reset_index(drop=True)

In [337]:
import re

def normalize_elongated_words(text):
    # Remove elongation
    text = re.sub(r'(.)\1+', r'\1\1', text)
    return text

df['review'] = df['review'].apply(normalize_elongated_words)


In [338]:
from nltk.corpus import stopwords
import re

arabic_stopwords = set(stopwords.words('arabic'))

def has_real_word(text):
    words = re.findall(r'[\u0600-\u06FF]+', text)  # Extract Arabic words
    return any(word in arabic_stopwords for word in words)

df = df[df['review'].apply(has_real_word)]

def is_gibberish(text):
    words = text.split()
    # If there are less than 2 words, or one very long "word" with no spaces
    return len(words) < 2 or max(len(w) for w in words) > 20

df = df[~df['review'].apply(is_gibberish)]


df = df.reset_index(drop=True)

In [339]:
df['rating'].value_counts()

rating
1    10573
5     9673
4     1613
3     1389
2      822
Name: count, dtype: int64

In [340]:
df['sentiment'].value_counts()

sentiment
positive    12675
negative    11395
Name: count, dtype: int64

In [341]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('arabic'))
def remove_stopwords(text):
    words = text.split()
    filtered = [w for w in words if w not in stopwords]
    return ' '.join(filtered)

df['clean_review'] = df['review'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mw865\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [342]:
df

,rating,review,sentiment,clean_review
0,1,سحبو للمره الثانيه رصيد من البطاقه و لغو الطلب...,negative,سحبو للمره الثانيه رصيد البطاقه لغو الطلب رجعو...
1,1,والله ما انصح بطلبات ما عندهم مصدقيه بالوقت بس...,negative,والله انصح بطلبات عندهم مصدقيه بالوقت البرنامج...
2,4,اوحش حاجه زياده الاسعار عن اماكن اخري,positive,اوحش حاجه زياده الاسعار اماكن اخري
3,5,اليوم قمت باجراء طلبين وتم سحب المبلغ والي الا...,positive,اليوم قمت باجراء طلبين وتم سحب المبلغ والي الا...
4,4,لوسمحت انا كنت منزال التطبيق علي تلفوني وضاع م...,positive,لوسمحت انا كنت منزال التطبيق علي تلفوني وضاع م...
...,...,...,...,...
24065,5,حييل سهل وحلو والمطاعم كثيره الله يعطيكم العاف...,positive,حييل سهل وحلو والمطاعم كثيره الله يعطيكم العاف...
24066,5,الصدق برنامج بقمت الروعه ومتعوب عليه يكفيكم ان...,positive,الصدق برنامج بقمت الروعه ومتعوب يكفيكم انه خدم...
24067,5,البرنامج خرافي ،وبصراحه تعليقات الاخوان سليمه ...,positive,البرنامج خرافي ،وبصراحه تعليقات الاخوان سليمه ...
24068,4,راءع جدا للمدن الكبيره، فقط لو يزيد عدد المطاع...,positive,راءع جدا للمدن الكبيره، فقط يزيد عدد المطاعم و...


In [343]:
df.to_csv(r'C:\Users\mw865\Desktop\NTI_Project\data\processed\cleaned_data.csv' , index=False , encoding='utf-8-sig')